## LangChain Agent, Tool, LLM

Based on:
- [Pinecone LangChain agens](https://www.pinecone.io/learn/series/langchain/langchain-agents/)

To use agents, we require three things:

- A base LLM
- A tool that we will be interacting with
- An agent to control the interaction.

### Auth

In [280]:
import google.auth
import os
import vertexai


from dotenv import load_dotenv, dotenv_values

In [281]:
langchain_api_key = os.environ["LANGCHAIN_API_KEY"] = os.getenv('LANGCHAIN_API_KEY')
langchain_project_id = os.getenv('LANGCHAIN_PROJECT_ID')
langchain_api_key

'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'

In [282]:
project_id = os.getenv('PROJECT_ID')
region_id = os.getenv('REGION_ID')
llm_model_id = os.getenv('CHAT_MODEL')
embedding_model_id = os.getenv('EMB_MODEL')

In [284]:
config = {
    'project_id': project_id,
    'llm_model_id': llm_model_id,
    'embedding_model_id': embedding_model_id
}

# authenticate to GCP
creds, _ = google.auth.default(quota_project_id=config["project_id"])
print(creds)

### Models

- The use of VertexAI instead of ChatVertexAI uses the [ReAct COT notebook example from GCP](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/prompts/examples/chain_of_thought_react.ipynb)

In [285]:
from langchain_google_vertexai import VertexAI, ChatVertexAI, VertexAIEmbeddings

vertexai.init(project=project_id, location=region_id)

# ChatVertexAI doesn't seem to work in the ReAct agent setting as well as VertexAI
# llm = ChatVertexAI(
#     credentials=creds,
#     model_name=config["llm_model_id"],
#     temperature=0 # to pick more exact answers for tools use-cases
# )

# VertexAI seems to work as well in the ReAct agent setting as ChatVertexAI
llm = VertexAI(
    credentials=creds,
    model_name=config["llm_model_id"],
    temperature=0
)

embeddings = VertexAIEmbeddings(
    credentials=creds, model_name=config["embedding_model_id"])


### Tracing

In [217]:
def trace(toggle):
    if toggle:
        os.environ['LANGCHAIN_TRACING_V2'] = 'true'
        os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
        os.environ['LANGCHAIN_API_KEY'] = langchain_api_key
        os.environ['LANGCHAIN_PROJECT'] = langchain_project_id
    else:
        del os.environ['LANGCHAIN_TRACING_V2']
        del os.environ['LANGCHAIN_ENDPOINT']
        del os.environ['LANGCHAIN_API_KEY']
        del os.environ['LANGCHAIN_PROJECT']

In [218]:
trace(True)

In [219]:
from langsmith import Client

client = Client(api_key=langchain_api_key)

url = next(client.list_runs(project_name="default")).url
print(f"You can monitor your trace at: {url}")

You can monitor your trace at: https://smith.langchain.com/o/2b56632f-de5b-5167-8339-f61d01e2ecb2/projects/p/3491b5eb-5738-4dbd-b4f1-bd7ed41971d4/r/0e27fd8d-0d4b-4a8f-b21f-51758f993bca?trace_id=94f473ad-0f02-44e5-a313-9bd4256709ae&start_time=2024-05-23T22:29:20.973000


### Calculator tool

- Uses [LLMMathChain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm_math.base.LLMMathChain.html): a chain that interprets a prompt and executes python code to do math.

In [220]:
from langchain.chains import LLMMathChain
from langchain.agents import Tool

llm_math = LLMMathChain.from_llm(llm)
llm_math.prompt.format(question="What is 2*4?")

'Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${Question with math problem.}\n```text\n${single line mathematical expression that solves the problem}\n```\n...numexpr.evaluate(text)...\n```output\n${Output of running the code}\n```\nAnswer: ${Answer}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: What is 2*4?\n'

In [221]:
llm_math.invoke("What is 2*4?")

{'question': 'What is 2*4?', 'answer': 'Answer: 8'}

In [222]:
calculator_tool = Tool(
    name='Calculator',
    func=llm_math.run, # .run on top of an existing chain
    description='A tool that is useful when the user needs answers to math questions.'
)

#### Add to list of tools

In [223]:
# when giving tools to LLM, we must pass as list of tools
tools = [calculator_tool]

### Agent

So far we have an llm and a tool but not an agent. To use agents, we require three things:

- A base LLM
- A tool that we will be interacting with
- An agent to control the interaction

#### Zero-shot agents
- Zero-shot means the agent functions on the current action only — it has no memory.
- It uses the ReAct framework to decide which tool to use, based solely on the tool’s `description`.
- The ReAct standard prompt is [hwchase17/react](https://smith.langchain.com/hub/hwchase17/react)
- ReAct can be thought about as an LLM that could cycle through Reasoning and Action steps. Enabling a multi-step process for identifying answers.

In [385]:
from langchain import hub
from langchain.agents import initialize_agent, AgentType

# this agent uses hwchase17/structured-chat-agent as the prompt
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [386]:
agent.invoke({'input': 'What is 17+32?'})



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Calculator",
  "action_input": "17+32"
}
```

Observation: Answer: 49
Thought:## Action:

```json
{
  "action": "Final Answer",
  "action_input": "The answer is 49."
}
```

## 

> Finished chain.


{'input': 'What is 17+32?', 'output': 'The answer is 49.'}

In [288]:
agent.invoke({
    "input": "if Mary has four apples and Giorgio brings two and a half apple " \
        "boxes (apple box contains eight apples), how many apples do we " \
            "have?"})



> Entering new AgentExecutor chain...
Action:
```json
{
  "action": "Calculator",
  "action_input": "4 + 2.5 * 8"
}
```
Observation: Answer: 24.0
Thought:Action:
```json
{
  "action": "Final Answer",
  "action_input": "Mary and Giorgio have 24 apples in total."
}
```

> Finished chain.


{'input': 'if Mary has four apples and Giorgio brings two and a half apple boxes (apple box contains eight apples), how many apples do we have?',
 'output': 'Mary and Giorgio have 24 apples in total.'}

In [244]:
agent.invoke({
    "input":"If I had 2 shoes and 4 socks and two socks are enough for one shoe, how many socks do I not need?"})



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Final Answer",
  "action_input": "You do not need 2 socks. You have 4 socks and you need 2 socks for each of your 2 shoes, so you need 4 socks in total. Therefore, you do not need any socks."
}
```


> Finished chain.


{'input': 'If I had 2 shoes and 4 socks and two socks are enough for one shoe, how many socks do I not need?',
 'output': 'You do not need 2 socks. You have 4 socks and you need 2 socks for each of your 2 shoes, so you need 4 socks in total. Therefore, you do not need any socks.'}

##### Non-math questions to agent

In [245]:
agent.invoke({'input':'What is the weather tomorrow?'})



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Final Answer",
  "action_input": "I am sorry, I cannot answer that question. I do not have access to real-time weather information."
}
```


> Finished chain.


{'input': 'What is the weather tomorrow?',
 'output': 'I am sorry, I cannot answer that question. I do not have access to real-time weather information.'}

### Fallback tool

In [246]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

fallback_llm_prompt = PromptTemplate(
    input_variables=["query"],
    template="{query}"
)

fallback_llm_chain = LLMChain(llm=llm, prompt=fallback_llm_prompt)

fallback_llm_tool = Tool(
    name='Fallback Tool',
    func=fallback_llm_chain.run,
    description="use this tool for general purpose queries and logic that aren't Medicare questions or math questions"
)

### Add to list of tools

In [247]:
tools = [
    calculator_tool,
    fallback_llm_tool
]

### Reinitialize agent

In [383]:
from langchain.agents import AgentExecutor, create_structured_chat_agent

# this agent uses hwchase17/structured-chat-agent as the prompt
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [384]:
agent.invoke({'input': 'What is the weather tomorrow?'})



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Fallback Tool",
  "action_input": "What is the weather tomorrow?"
}
```
Observation: I do not have access to real-time weather information. To get the most up-to-date weather forecast for your location, I recommend checking a reliable weather app or website. 

Would you like me to try and find a weather forecast for your location from a different source? 

Thought:Action:
```
{
  "action": "Final Answer",
  "action_input": "I do not have access to real-time weather information. To get the most up-to-date weather forecast for your location, I recommend checking a reliable weather app or website. 

Would you like me to try and find a weather forecast for your location from a different source? "
}
```


> Finished chain.


{'input': 'What is the weather tomorrow?',
 'output': 'I do not have access to real-time weather information. To get the most up-to-date weather forecast for your location, I recommend checking a reliable weather app or website. \n\nWould you like me to try and find a weather forecast for your location from a different source? '}

#### Ask fallback questions to agent

In [290]:
agent.invoke({'input': 'What is the weather tomorrow?'})



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Fallback Tool",
  "action_input": "What is the weather tomorrow?"
}
```
Observation: I do not have access to real-time weather information. To get the most accurate and up-to-date weather forecast for tomorrow, I recommend checking a reliable weather source such as the National Weather Service or a local news station. 

Would you like me to try and find the weather forecast for you from another source? 

Thought:Action:
```
{
  "action": "Fallback Tool",
  "action_input": "What is the weather tomorrow?"
}
```
Observation: I do not have access to real-time information, including weather forecasts. To get the most accurate and up-to-date weather information for tomorrow, I recommend checking a reliable weather source such as the National Weather Service or a local news station. 

Would you like me to try and find the weather forecast for you from another source? 

Thought:Action:
```
{
  "action": "Fallback Tool",
  "acti

{'input': 'What is the weather tomorrow?',
 'output': 'I am sorry, I do not have access to real-time information, including weather forecasts. To get the most accurate and up-to-date weather information for tomorrow, I recommend checking a reliable weather source such as the National Weather Service or a local news station.'}

#### Do math questions still work?

In [291]:
agent.invoke({'input':'What is seventeen to the power of 3?'})



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Calculator",
  "action_input": "17^3"
}
```

Observation: Answer: 4913
Thought:Action:
```
{
  "action": "Final Answer",
  "action_input": "17 to the power of 3 is 4913."
}
```

> Finished chain.


{'input': 'What is seventeen to the power of 3?',
 'output': '17 to the power of 3 is 4913.'}

#### How about combination math and general questions?

In [292]:
agent.invoke({'input':'A plant can grow 6 fruit per day. How many fruits grow in 30 days? What is this plant called?'})



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Calculator",
  "action_input": "6 * 30"
}
```

Observation: Answer: 180
Thought:Action:
```
{
  "action": "Fallback Tool",
  "action_input": "What is this plant called?"
}
```

Observation: I am sorry, but I cannot answer your question without an image of the plant. Please provide an image of the plant you are referring to, and I will do my best to identify it for you.
Thought:Action:
```
{
  "action": "Final Answer",
  "action_input": "A plant can grow 180 fruits in 30 days. I am sorry, but I cannot answer your question about the name of the plant without an image of the plant. Please provide an image of the plant you are referring to, and I will do my best to identify it for you."
}
```

> Finished chain.


{'input': 'A plant can grow 6 fruit per day. How many fruits grow in 30 days? What is this plant called?',
 'output': 'A plant can grow 180 fruits in 30 days. I am sorry, but I cannot answer your question about the name of the plant without an image of the plant. Please provide an image of the plant you are referring to, and I will do my best to identify it for you.'}

### Medicare tool

#### Data

In [293]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma 
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_vertexai import VertexAIEmbeddings

loader = PyPDFLoader("https://www.medicare.gov/Pubs/pdf/10050-medicare-and-you.pdf", extract_images=True)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
db = Chroma.from_documents(documents, embeddings)

#### Retriever

In [294]:
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain.schema import StrOutputParser

retriever = db.as_retriever(search_kwargs={'k':1}) # just one doc

#### Tool

In [295]:
from langchain.tools.retriever import create_retriever_tool

medicare_tool = create_retriever_tool(
    retriever,
    "Medicare Knowledge Base",
    "Query a retriever to get information about Medicare questions"
)

In [296]:
medicare_tool.invoke('What is Medicare part D?')

'9\nWhat are the parts of \nMedicare?\nPart A (Hospital Insurance) \nHelps cover:\n• Inpatient care in hospitals\n• Skilled nursing facility care\n• Hospice care\n• Home health care\nGo to pages 25–29. \nPart B (Medical Insurance)\nHelps cover:\n• Services from doctors and other health care providers\n• Outpatient care\n• Home health care\n• Durable medical equipment (like wheelchairs, walkers, \nhospital beds, and other equipment)\n• Many preventive services  (like screenings, shots or vaccines, \nand yearly “ Wellness” visits) \nGo to pages 29–55 . \nPart D (Drug coverage) \nHelps cover the cost of prescription drugs (including many recommended shots or vaccines).\nPlans that offer Medicare drug coverage (Part D) are run by \nprivate insurance companies that follow rules set by Medicare.\nGo to pages 79–90.'

#### Structured output

- This is important to avoid future steps where agent needs to take directed action
- We want the final answer to have two fields: one for the answer, and then another that is a list of sources
- TODO

In [266]:
from typing import List

from langchain_core.pydantic_v1 import BaseModel, Field

### Add to list of tools

In [297]:
tools = [
    calculator_tool,
    fallback_llm_tool,
    medicare_tool
]

### Reinitialize agent

In [339]:
# this agent uses hwchase17/structured-chat-agent as the prompt
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

#### Ask medicare questions to agent

In [340]:
agent.invoke({'input': 'What is Medicare part A?'})



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Medicare Knowledge Base",
  "action_input": {
    "query": "What is Medicare Part A?"
  }
}
```
Observation: 9
What are the parts of 
Medicare?
Part A (Hospital Insurance) 
Helps cover:
• Inpatient care in hospitals
• Skilled nursing facility care
• Hospice care
• Home health care
Go to pages 25–29. 
Part B (Medical Insurance)
Helps cover:
• Services from doctors and other health care providers
• Outpatient care
• Home health care
• Durable medical equipment (like wheelchairs, walkers, 
hospital beds, and other equipment)
• Many preventive services  (like screenings, shots or vaccines, 
and yearly “ Wellness” visits) 
Go to pages 29–55 . 
Part D (Drug coverage) 
Helps cover the cost of prescription drugs (including many recommended shots or vaccines).
Plans that offer Medicare drug coverage (Part D) are run by 
private insurance companies that follow rules set by Medicare.
Go to pages 79–90.
Thought:Action:
```
{
  "act

{'input': 'What is Medicare part A?',
 'output': 'Medicare Part A is the part of Medicare that helps cover inpatient hospital care, skilled nursing facility care, hospice care, and home health care. It is a valuable resource for people who need these types of care.'}

#### Do math questions still work?

In [341]:
agent.invoke({'input': 'What is 17+4?'})



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Calculator",
  "action_input": "17+4"
}
```
Observation: Answer: 21
Thought:Action:
```
{
  "action": "Final Answer",
  "action_input": "The answer is 21."
}
```


> Finished chain.


{'input': 'What is 17+4?', 'output': 'The answer is 21.'}

#### How about combination math and general questions?

In [342]:
agent.invoke({'input': 'What is the capital of Hungary? What is 99 minus 23??'})



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Fallback Tool",
  "action_input": "What is the capital of Hungary?"
}
```

Observation: The capital of Hungary is Budapest.
Thought:Action:
```
{
  "action": "Calculator",
  "action_input": "99 - 23"
}
```
Observation: Answer: 76
Thought:Action:
```
{
  "action": "Final Answer",
  "action_input": "The capital of Hungary is Budapest. 99 minus 23 is 76."
}
```

> Finished chain.


{'input': 'What is the capital of Hungary? What is 99 minus 23??',
 'output': 'The capital of Hungary is Budapest. 99 minus 23 is 76.'}

In [343]:
agent.invoke({'input': 'I have a son aged 10 years. I have a daughter that is 20 years older than the son. What Medicare benefits do my two dependents enjoy?'})



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Medicare Knowledge Base",
  "action_input": {
    "query": "What Medicare benefits do dependents enjoy?"
  }
}
```
Observation: SECTION 2:  Find out what Medicare covers27
Medicare also covers inpatient respite care, which is care you get in a Medicare-
approved facility so that your usual caregiver (family member or friend) can rest. You can stay up to 5 days each time you get respite care. 
After 6 months, you can continue to get hospice care as long as the hospice 
medical director or hospice doctor recertifies (at a face-to-face meeting) that you’re still terminally ill. 
You pay:
• Nothing for hospice care. 
• A copayment  of up to $5 per prescription for outpatient drugs for pain and 
symptom management. 
• Five percent of the Medicare-approved amount  for inpatient respite care. 
Original Medicare will be billed for your hospice care, even if you’re in a Medicare Advantage Plan . When you get hospice care, your M

{'input': 'I have a son aged 10 years. I have a daughter that is 20 years older than the son. What Medicare benefits do my two dependents enjoy?',
 'output': "I'm sorry, but I can't answer that question. Medicare benefits for dependents are complex and depend on a variety of factors, including the age of the dependent, their relationship to the beneficiary, and their disability status. I recommend that you contact Medicare directly for more information."}

### Agent with Memory
- According to [LangChain AgentTypes](https://python.langchain.com/v0.1/docs/modules/agents/agent_types/) we need an agent that can support both:
  - History
  - Multi tools
- That seems to be [Structured Chat](https://python.langchain.com/v0.1/docs/modules/agents/agent_types/structured_chat/)

#### Memory

In [344]:
memory_key='chat_history'

In [356]:
from langchain.memory import ConversationBufferWindowMemory

agent_memory = ConversationBufferWindowMemory(
    memory_key=memory_key,
    k=5,
    return_messages=True
)

#### Agent

In [355]:
from langchain import hub

prompt = hub.pull("hwchase17/structured-chat-agent")

In [371]:
from langchain.agents import create_structured_chat_agent, AgentExecutor


memory_agent = create_structured_chat_agent(llm, tools, prompt)
memory_agent_executor = AgentExecutor(
    agent=memory_agent, 
    tools=tools, 
    verbose=True, 
    memory=agent_memory,
    return_intermediate_steps=True, 
    handle_parsing_errors=True
)

In [372]:
memory_agent_executor.invoke({"input": "My name is Bob!"})



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Final Answer",
  "action_input": "Hi Bob, it's nice to meet you!"
}
```


> Finished chain.


/opt/homebrew/Caskroom/miniforge/base/lib/python3.11/site-packages/langchain/memory/chat_memory.py:34: UserWarning: 'ConversationBufferWindowMemory' got multiple output keys: dict_keys(['output', 'intermediate_steps']). The default 'output' key is being used. If this is not desired, please manually set 'output_key'.
  warnings.warn(


{'input': 'My name is Bob!',
 'chat_history': [HumanMessage(content='Would that be enough money to pay for a busride in the city I live in?'),
  AIMessage(content="I'm sorry, I don't have enough information to answer that question. Would you like to tell me the cost of a bus ride in your city?"),
  HumanMessage(content='I live in NYC'),
  AIMessage(content="I'm sorry, I don't have enough information to answer that question. Would you like to tell me the cost of a bus ride in your city?")],
 'output': "Hi Bob, it's nice to meet you!",
 'intermediate_steps': []}

In [373]:
memory_agent_executor.invoke({"input": "What is my name?"})



> Entering new AgentExecutor chain...
## Action:

```json
{
  "action": "Final Answer",
  "action_input": "Your name is Bob."
}
``` 


> Finished chain.


/opt/homebrew/Caskroom/miniforge/base/lib/python3.11/site-packages/langchain/memory/chat_memory.py:34: UserWarning: 'ConversationBufferWindowMemory' got multiple output keys: dict_keys(['output', 'intermediate_steps']). The default 'output' key is being used. If this is not desired, please manually set 'output_key'.
  warnings.warn(


{'input': 'What is my name?',
 'chat_history': [HumanMessage(content='Would that be enough money to pay for a busride in the city I live in?'),
  AIMessage(content="I'm sorry, I don't have enough information to answer that question. Would you like to tell me the cost of a bus ride in your city?"),
  HumanMessage(content='I live in NYC'),
  AIMessage(content="I'm sorry, I don't have enough information to answer that question. Would you like to tell me the cost of a bus ride in your city?"),
  HumanMessage(content='My name is Bob!'),
  AIMessage(content="Hi Bob, it's nice to meet you!")],
 'output': 'Your name is Bob.',
 'intermediate_steps': []}